In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import json
import plotly.express as px

In [2]:
df = pd.read_csv('berlin_weather_forecast.csv') 
df['date'] = pd.to_datetime(df['date'].astype(str))
df['year'] = df['date'].dt.year
df_gb = df.groupby(['name','year','week'])['predicted_tempreature'].mean().reset_index()
df_gb['predicted_tempreature'] = df_gb['predicted_tempreature'].round(2)
df_gb['predicted_tempreature'] = df_gb['predicted_tempreature']*100
#df_gb.head()
df_gb['predicted_tempreature'].min(), df_gb['predicted_tempreature'].max()

(150.0, 2195.0)

In [3]:
filename = "berlin_bezirke.geojson"
file = open(filename)
geo_json_file = gpd.read_file(file)
#type(geo_json_file)
json_file = geo_json_file.to_json()
converted_json = json.loads(json_file)
#converted_json

In [4]:
interactive_map = px.choropleth_mapbox(
    mapbox_style='open-street-map',      # see other styles: https://plotly.com/python/mapbox-layers/
    data_frame = df_gb,                  # dataframe that contains all years
    geojson=converted_json,                   # we can still use the JSON data from 2000, assuming the countries are the same over time
    featureidkey='properties.name',      # name of JSON key within the "properties" value that contains country names
    locations='name',                    # name of the dataframe column that contains country names
    color='predicted_tempreature',                       # name of the dataframe column that contains numerical data you want to display
    center={"lat": 52.5200, "lon": 13.4050},
    zoom=8,
    animation_frame='week',             # name of dataframe column that you want to make frames of
    animation_group='name',   
    color_continuous_scale="thermal",    # see other options: https://plotly.com/python/builtin-colorscales/
    range_color=(-100, 2250),
    color_continuous_midpoint=1100)

In [5]:
interactive_map.write_html("berlin_interactive_map.html", include_plotlyjs='cdn')